# HSE 2021: Mathematical Methods for Data Analysis

## Homework 4

**Warning 1**: You have 2 weeks for this assignemnt.  **it is better to start early (!)**

**Warning 2**: it is critical to describe and explain what you are doing and why, use markdown cells


### Contents

#### Decision Trees - 7 points
* [Task 1](#task1) (0.5 points)
* [Task 2](#task2) (0.5 points)
* [Task 3](#task3) (2 points)
* [Task 4](#task4) (0.5 points)
* [Task 5](#task5) (0.5 points)
* [Task 6](#task6) (2 points)
* [Task 7](#task7) (0.5 points)
* [Task 8](#task8) (0.5 points)

#### Ensembles - 3 points
* [Task 1](#task2_1) (1 point)
* [Task 2](#task2_2) (0.7 points)
* [Task 3](#task2_3) (0.5 points)
* [Task 4](#task2_4) (0.7 points)
* [Task 5](#task2_5) (0.1 points)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

plt.rcParams['figure.figsize'] = (11, 5)
%matplotlib inline

# Part 1. Decision Tree Regressor

In this task you will be implementing decision tree for the regression by hand. 

### Task 1 <a id="task1"></a> (0.5 points)

Here you should implement the function `H()` which calculates impurity criterion. We will be training regression tree, and will take mean absolute deviation as impurity criterion.

* You cannot use loops
* If `y` is empty, the function should return 0

In [ ]:
def H(y):
    """
    Calculate impurity criterion
    
    Parameters
    ----------
    y : np.array
        array of objects target values in the node

    Returns
    -------
    H(R) : float
        Impurity in the node (measuread by variance)
    """
    if y.size == 0:
        return 0
    # https://www.cuemath.com/variance-formula/
    return np.sum(np.square(y - np.sum(y) / y.size)) / y.size

In [ ]:
# Test the function
assert np.allclose(H(np.array([4, 2, 2, 2])), 0.75)
assert np.allclose(H(np.array([])), 0.0)

### Task 2 <a id="task2"></a>  (0.5 points)

To find the best split in the node we need to calculate the cost function. Denote: 
- `R` all the object in the node
- `j` index of the feature selected for the split
- `t` threshold
- `R_l` and `R_r` objects in the left and right child nodes correspondingly

We get the following cost function:

$$
Q(R, j, t) =\frac{|R_\ell|}{|R|}H(R_\ell) + \frac{|R_r|}{|R|}H(R_r) \to \min_{j, t},
$$

Implement the function `Q`, which should calculate value of the cost function for a given feature and threshold.

In [ ]:
def Q(X, y, j, t):
    """
    Calculate cost function
    Parameters
    ----------
    X : ndarray
        array of objects in the node 
    y : ndarray
        array of target values in the node 
    j : int
        feature index (column in X)
    t : float
        threshold

    Returns
    -------
    Q : float
        Value of the cost function
    """
    if y.size == 0:
        return 0

    # Это с семинара
    # https://github.com/darkydash/ml_hse/blob/main/week07/07_HSE_SE_DT_solved.ipynb
    # class DecisionTreeClassifier_from_scratch
    # Возьмем наилучшее разделение
    x_col = X[:, j]
    y_left = y[x_col <= t]
    y_right = y[x_col > t]   

    # Посчитаем итоговое значение cost function
    return partial_cost(y, y_right) + partial_cost(y, y_left)


def partial_cost(whole, part):
    return part.size / whole.size * H(part)

### Task 3 <a id="task3"></a>  (2 points)

Now, let's implement `MyDecisionTreeRegressor` class. More specifically, you need to implement the following methods:

- `best_split`
- `grow_tree`
- `get_prediction`

Also, please add `min_samples_leaf` parameter to your class

Read docstrings for more details. Do not forget to use function `Q` implemented above, when finding the `best_split`

In [ ]:
class Node(object):
    """
    Class for a decision tree node.
    
    Parameters
    ----------
    right : Node() or None
        Right child
    right : Node() or None
        Left child
    threshold: float
        
    column: int
        
    depth: int
        
    prediction: float
        prediction of the target value in the node 
        (average values calculated on a train dataset)
    is_terminal:bool
        indicates whether it is a terminal node (leaf) or not
    """    
    def __init__(self):        
        self.right = None
        self.left = None
        self.threshold = None
        self.column = None
        self.depth = None
        self.is_terminal = False
        self.prediction = None
        
    def __repr__(self):
        if self.is_terminal:
            node_desc = 'Pred: {:.2f}'.format(self.prediction)
        else:
            node_desc = 'Col {}, t {:.2f}, Pred: {:.2f}'. \
            format(self.column, self.threshold, self.prediction)
        return node_desc

In [ ]:
from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted

class MyDecisionTreeRegressor(RegressorMixin, BaseEstimator):
    """
    Class for a Decision Tree Regressor.

    Parameters
    ----------
    max_depth : int
        Max depth of a decision tree.
    min_samples_split : int
        Minimal number of samples (objects) in a node to make a split.
    """
    def __init__(self, max_depth = 3, min_samples_split = 2, min_samples_leaf = 1):
        self.max_depth = max_depth
        self.min_samples_split = min_samples_split
        # min_samples_leaf уже был добавлен, хоть в условии его просят добавить
        self.min_samples_leaf = min_samples_leaf
            
    def best_split(self, X, y):
        """
        Find the best split in terms of Q of data in a given decision tree node. 
        Try all features and thresholds. 

        Parameters
        ----------
        X : ndarray, shape (n_objects, n_features)
            Objects in the parent node
        y : ndarray, shape (n_objects, )
            1D array with the object labels. 

        Returns
        -------
        best_split_column : int
            Index of the best split column
        best_threshold : float
            The best split condition.
        X_left : ndarray, shape (n_objects_l, n_features)
            Objects in the left child
        y_left : ndarray, shape (n_objects_l, )
            Objects labels in the left child. 
        X_right : ndarray, shape (n_objects_r, n_features)
            Objects in the right child
        y_right : ndarray, shape (n_objects_r, )
            Objects labels in the right child. 
        """
        # To store best split parameters
        best_split_column = None
        best_threshold = None
        # without splitting
        best_cost = H(y)
        best_flag = False
        X_left, y_left, X_right, y_right = None, None, None, None

        # Цикл перебора индексов i, j
        # На самом деле перебор параметров j, t для функции Q
        # С целью минимизации значения данной функции (как показано в формуле)
        for j in range(X.shape[1]):
            for index in range(X[:,j].size):
                current_cost = Q(X, y, j, X[index, j])
                if current_cost < best_cost:
                    best_split_column = j
                    best_threshold = X[index, j]
                    best_cost = current_cost
                    best_flag = True

        if best_flag:
            # Это с семинара
            # https://github.com/darkydash/ml_hse/blob/main/week07/07_HSE_SE_DT_solved.ipynb
            # class DecisionTreeClassifier_from_scratch
            x_col = X[:, best_split_column]
            X_left = X[x_col <= best_threshold, :]
            y_left = y[x_col <= best_threshold]
            X_right = X[x_col > best_threshold, :]
            y_right = y[x_col > best_threshold]

        return best_split_column, best_threshold, X_left, y_left, X_right, y_right
    
    def is_terminal(self, node, y):
        """
        Check terminality conditions based on `max_depth`, 
        `min_samples_split` and `min_samples_leaf` parameters for a given node. 

        Parameters
        ----------
        node : Node, 

        y : ndarray, shape (n_objects, )
            Object labels. 

        Returns
        -------
        Is_termial : bool
            If True, node is terminal
        """
        # Поменял на нестрогое, так как счетчик начинается с 1
        if node.depth > self.max_depth:
            return True
        if len(y) < self.min_samples_split:
            return True
        return False
        
    def grow_tree(self, node, X, y):
        """
        Reccurently grow the tree from the `node` using a `X` and `y` as a dataset:
         - check terminality conditions
         - find best split if node is not terminal
         - add child nodes to the node
         - call the function recursively for the added child nodes

        Parameters
        ----------
        node : Node() object
            Current node of the decision tree.
        X : ndarray, shape (n_objects, n_features)
            Objects 
        y : ndarray, shape (n_objects)
            Labels
        """
        if self.is_terminal(node, y):
            node.is_terminal = True
            return
        
        # Общая логика метода взята с семинара
        # https://github.com/darkydash/ml_hse/blob/main/week07/07_HSE_SE_DT_solved.ipynb
        # Клетка 10, def decision_tree
        if X.size < self.min_samples_split:   # min_samples_split check
            node.is_terminal = True
            return

        # Вызов best_split, написанного ранее, для получения наилучшего разделения (с минимальной "ценой")
        best_split_column, best_threshold, X_left, y_left, X_right, y_right = self.best_split(X, y)

        # Check additional termination conditions
        # https://github.com/darkydash/ml_hse/blob/main/week07/07_HSE_SE_DT_solved.ipynb
        if X_right is None:
            node.is_terminal = True
            return
        if X_right.size < self.min_samples_leaf or X_left.size < self.min_samples_leaf:  # min_samples_leaf check
            node.is_terminal = True
            return

        # "Проростание" двух вершин из текущей
        node.right, node.left = Node(), Node()

        # Заполнение оставшихся значений для текущей вершины
        node.column = best_split_column
        node.threshold = best_threshold

        # Заполнение части значений для новых вершин
        self.fill_node(node.right, node.depth, y_right)
        self.fill_node(node.left, node.depth, y_left)

        # Рекурсивный рост дерева
        self.grow_tree(node.right, X_right, y_right)
        self.grow_tree(node.left, X_left, y_left)

    # Сделано по аналогии метода fit()
    # Заполнение части значений для новых вершин
    def fill_node(self, partial_node, depth, previous):
        partial_node.depth = depth + 1
        partial_node.prediction = np.mean(previous)

    def fit(self, X, y):
        """
        Fit the Decision Tree Regressor.

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            The input samples.
        y : ndarray, shape (n_samples,) or (n_samples, n_outputs)
            The target values.
        Returns
        -------
        self : object
            Returns self.
        """
        X, y = check_X_y(X, y, accept_sparse = False)
        self.is_fitted_ = True

        # Initialize the tree (root node)
        self.tree_ = Node()
        self.tree_.depth = 1
        self.tree_.prediction = np.mean(y)

        # Grow the tree
        self.grow_tree(self.tree_, X, y)
        return self       
    
    def get_prediction(self, node, x):
        """
        Get prediction for an object `x`
            - Return prediction of the `node` if it is terminal
            - Otherwise, recursively call the function to get 
            predictions of the proper child

        Parameters
        ----------
        node : Node() object
            Current node of the decision tree.
        x : ndarray, shape (n_features,)
            Array of feature values of one object.
        Returns
        -------
        y_pred : float
            Prediction for an object x
        """
        # Конец вычисления
        if node.is_terminal:   
            return node.prediction              

        # Выбор оптимальной вершины (левой или правой)
        optimal_node = node.left
        if x[node.column] > node.threshold:
            optimal_node = node.right

        # Рекурсивное вычисление (шагаем до листа)
        return self.get_prediction(optimal_node, x)

    def predict(self, X):
        """ 
        Get prediction for each object in X

        Parameters
        ----------
        X : ndarray, shape (n_samples, n_features)
            The input samples.
        Returns
        -------
        y : ndarray, shape (n_samples,)
            Returns predictions.
        """
        # Check input and that `fit` had been called
        X = check_array(X, accept_sparse = False)
        check_is_fitted(self, 'is_fitted_')

        # Get predictions
        y_predicted = []
        for x in X:
            y_curr = self.get_prediction(self.tree_, x)
            y_predicted.append(y_curr)
        return np.array(y_predicted)

In [ ]:
# check yourself
from sklearn.utils.estimator_checks import check_estimator

check_estimator(MyDecisionTreeRegressor())

/usr/local/lib/python3.7/dist-packages/sklearn/utils/estimator_checks.py:3617: FutureWarning: As of scikit-learn 0.23, estimators should expose a n_features_in_ attribute, unless the 'no_validation' tag is True. This attribute should be equal to the number of features passed to the fit method. An error will be raised from version 1.0 (renaming of 0.25) when calling check_estimator(). See SLEP010: https://scikit-learn-enhancement-proposals.readthedocs.io/en/latest/slep010/proposal.html
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/utils/estimator_checks.py:3652: FutureWarning: As of scikit-learn 0.23, estimators should have a 'requires_y' tag set to the appropriate value. The default value of the tag is False. An error will be raised from version 1.0 when calling check_estimator() if the tag isn't properly set.
  warnings.warn(warning_msg, FutureWarning)


Появилось два warning, но я так понимаю не подразумевается, что мы будем их исправлять. Раз ячейка отработала, значик все ок.

По поводу описания написанного кода: каждое действие прокомментировал в самом коде (методы best_split(), grow_tree(), get_prediction()).

Best_split - ищет разбиение с наименьшой "ценой".

Grow_tree (рекурсивная) - растит дерево, как раз используя best_split.

Get_prediction (рекурсивная) - пробегают по дереву (не по всему, а как бы по высоте дерева) и приходят в лист, из которого берется prediction.

### Task 4 <a id="task4"></a>  (0.5 points)

Load boston dataset and split it on the train ($75\%$) and test ($25\%$). Fit Decision Tree of depth 1 and make the following plot:

- Scatter plot of the traning points (selected for split feature on the x-axis, target variable on the y-axis)
- Fitted model 

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split


# Клетка 25
# https://github.com/darkydash/ml_hse/blob/main/week07/07_HSE_SE_DT_solved.ipynb
def plot_graphic(values_x_1, values_y_1, values_x_2, values_y_2):
    plt.figure()
    plt.scatter(values_x_1, values_y_1, label = 'Train')      
    plt.scatter(values_x_2, values_y_2, label = 'Prediction')
    plt.xlabel('Split feature') # set up X-axis label
    plt.ylabel('Target variable') # set up Y-axis label
    plt.legend(loc='best') # create the plot legend and set up it position
    plt.grid(b=1) # create grid on the plot
    plt.show() # display the plot

# Загрузка данных
data = load_boston()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size = 0.25, random_state = 18293)

# Обучение дерева
tree = MyDecisionTreeRegressor(max_depth = 1)
tree.fit(X_train, y_train)
prediction = tree.predict(X_test)

# Рисование графики
plot_graphic(X_train[:, tree.tree_.column], y_train, X_test[:, tree.tree_.column], prediction)

### Task 5 <a id="task5"></a>  (0.5 points)

Keep working with boston dataset. 
- Use `GridSearchCV` to find the best hyperparameters among [`max_depth`, `min_samples_leaf`] on 5-Fold cross-validation
- Train the model with the best set of hyperparameters on the whole train dataset. 
- Report `MAE` on test dataset and hyperparameters of the best estimator. 

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error

# Ввод values, по которым будет поиск
max_depth_values = [1, 5, 10, 15, 20, 25, 30]
min_samples_leaf_values = [1, 5, 10, 15, 20, 25, 30, 35, 40]

# Поиск наилучших параметров по метрике neg_mean_squared_error
# При выборе метрики опирался на семинар
# Там использованы accuracy и neg_mean_squared_error
# Но при использовнии accuracy здесь, на каждой итерации выдается warning
searcher = GridSearchCV(MyDecisionTreeRegressor(), {"max_depth": max_depth_values, "min_samples_leaf" : min_samples_leaf_values}, scoring = "neg_mean_squared_error", cv = 5)
searcher.fit(X_train, y_train)
best_param = searcher.best_params_

# Обучение
tree = MyDecisionTreeRegressor(max_depth = best_param["max_depth"], min_samples_leaf = best_param["min_samples_leaf"])
tree.fit(X_train, y_train)
prediction = tree.predict(X_test)

# Вывод полученных значений
MAE = mean_absolute_error(y_test, prediction)
print(f"MAE = {MAE}")
print(f"Лучшие параметры = {best_param}")

MAE = 3.1734786168970253
Лучшие параметры = {'max_depth': 15, 'min_samples_leaf': 15}


### Task 6 <a id="task6"></a>  (2 points)

Recall definition of bias and variance:
$$
\text{Bias}^2 = \mathbb{E}_{p(x, y)} \left[  (f(x) - \mathbb{E}_{\mathbb{X}}a_{\mathbb{X}}(x))^2 \right] \\
\text{Variance} = \mathbb{E}_{p(x, y)} \left[  \mathbb{V}_{\mathbb{X}}( a_{\mathbb{X}}(x))  \right]
$$

We wil now use use the following algorithm to estimate bias and variance:

1. Use bootsrap to create `n_iter` samples from the original dataset: $X_1, \dots, X_{n_iter}$
2. For each bootstrapped sample define out-of-bag (OOB) sample $Z_1, \dots, Z_{n_iter}$, which contain all the observations, which did not appear in the corresponding boostraped sample
3. Fit the model on $X_i$s and compute predictions on $Z_i$s
4. For a given *object* $n$:
     - bias^2: squared difference between true value $y_n$ and average prediction (average over the algorithms, for which $n$ was in OOB)
     - variance: variance of the prediction (predictions of the algorithms, for which $n$ was in OOB)
5. Average bias^2 and variance over all the points
    
**Implement `get_bias_variance` function, using the algorithm above**

*Note:*  You can only use 1 loop (for bootsrap iterations). All other operations should be vectorized. 

In [ ]:
import random

def get_bias_variance(estimator, x, y, n_iter):
    """ 
    Calculate bias and variance of the `estimator`. 
    Using a given dataset and bootstrap with `n_iter` samples. 

    Parameters
    ----------
    x : ndarray, shape (n_samples, n_features)
        The input samples.
    y : ndarray, shape (n_samples, n_features)
        The input samples.
    n_iter: int
        Number of samples in 
    Returns
    -------
    bias2 : float, 
        Estiamted squared bias
    variance : float, 
        Estiamted variance
    """
    random.seed(1723987) # сид рандома
    all_idx = np.array(range(y.size)) # просто все индексы y
    bias = np.zeros(n_iter) # массив, куда будем сохранять bias каждой итерации
    variance = np.zeros(n_iter) # массив, куда будем сохранять variance каждой итерации
    
    # https://medium.com/swlh/bootstrap-sampling-using-pythons-numpy-85822d868977
    for i in range(n_iter):
        # 1. Create n_iter samples from the original dataset
        # https://docs-python.ru/standart-library/modul-random-python/funktsija-random-choices/
        boot = random.choices(all_idx, k = all_idx.size)
        # Аналог разделения по threshold с того же семинара
        X_train_local, y_train_local = x[boot, :], y[boot]

        # 2. For each bootstrapped sample define out-of-bag (OOB) sample
        # https://stackoverflow.com/questions/55573872/how-to-subset-numpy-array-with-exclusion
        oob = all_idx[np.isin(all_idx, boot, invert = True)]
        X_test_local, y_test_local = x[oob, :], y[oob]

        # 3. Fit the model and compute predictions
        estimator.fit(X_train_local, y_train_local)
        pred_local = estimator.predict(X_test_local)

        # 4. Calculate and save bias^2 and variance
        # https://www.cuemath.com/variance-formula/
        bias[i] = np.mean(np.square(y_test_local - np.mean(pred_local)))
        variance[i] = np.sum(np.square(pred_local - np.sum(pred_local) / pred_local.size)) / pred_local.size
    
    # 5. Average bias^2 and variance over all the points
    return np.mean(bias), np.mean(variance)

In [ ]:
# Test
estimator = MyDecisionTreeRegressor(max_depth=8, min_samples_split=15)

# get_bias_variance(estimator, X_train, y_train, 10)
get_bias_variance(estimator, X_train, y_train, 10)

(83.24794232096762, 82.35336205387762)

### Task 7 <a id="task7"></a>  (0.5 points)

Compute bias and variance for the trees with different min_samples_split. Plot how bias and variance change as min_samples_split increases. 

Comment on what you observe, how does your result correspond to theory?

In [ ]:
# Массивы для bias, variance
biases, variances = [], []

# Просчет bias, variance для каждого значения min_samples_split
min_samples_split = [1, 3, 5, 7, 10, 13, 15, 17, 20]
for min_sample_split in min_samples_split:
    estimator = MyDecisionTreeRegressor(max_depth = 10, min_samples_split = min_sample_split)
    bias, variance = get_bias_variance(estimator, X_train, y_train, 10)
    # Сохранения bias, variance
    biases.append(bias)
    variances.append(variance)

# Рисования графика
plt.plot(min_samples_split, biases, label = "Biases")
plt.plot(min_samples_split, variances, label = "Variances")
plt.xlabel("Min samples split")
plt.ylabel("Bias / variance")
plt.legend()
plt.show()

Как мы знаем, bias - метрика, считающая системную ошибку предположения в процессе машинного обучения. Variance - это изменчивость формы целевой функции относительно различных обучающих наборов. Модели с высокой дисперсией могут пострадать даже при небольших изменениях в обучающем наборе. DecisionTreeRegressor очень чувствителен к различиям в данных.

Не смотря на то, что один из графиков визуально прыгает, на самом деле оба графики практически горизонтальны, ведь все значения находятся в диапазоне от 82 до 84. В данном случае min_sample_split практически не влияет на bias / variance.


### Task 8 <a id="task8"></a>  (0.5 points)

Let's try to reduce variance with bagging. Use `sklearn.ensemble.BaggingRegressor` to get an ensemble and compute its bias and variance. 

Answer the following questions:
 - How bagging should affect bias and variance in theory?
 - How bias and variance change (if they change) compared to an individual tree in you experiments? 
 - Do your results align with the theory? Why?

In [ ]:
from sklearn.ensemble import BaggingRegressor

regressor = BaggingRegressor(MyDecisionTreeRegressor(max_depth = 10, min_samples_split = 10), random_state = 1723987)
bias_bagging, variance_bagging = get_bias_variance(regressor, X_train, y_train, 10)
print(f"Bias = {bias_bagging}")
print(f"Variance = {variance_bagging}")

Bias = 83.03033198426117
Variance = 71.00277606100961


**How bagging should affect bias and variance in theory?**

 Не затронуть bias

 Уменьшить variance

**How bias and variance change (if they change) compared to an individual tree in you experiments?**

Bias не был затронут

Variance уменьшился

**Do your results align with the theory? Why?**

Да, в точности сошлись с теорией
